
# Vision Transformer with Mixed Precision on AMD Instinct MI300X 💻⚡

Welcome to my personal deep dive into Vision Transformers (ViTs)! I’ve always been intrigued by how Transformers—originally famous for NLP—can excel in computer vision tasks. In this notebook, I’m training a custom ViT on CIFAR-10, using **mixed precision** on an **AMD Instinct MI300X** GPU. 🎉

---

## Why This Project? 🤔

I’ve worked with CNNs for image tasks, but Transformers kept stealing the spotlight—especially since the original "Attention is All You Need" paper. I wanted to experience:

1. **RandAugment** 🌟: a super-quick trick for stronger data augmentation on CIFAR-10.
2. **Mixed Precision** 🚀: to speed up training on AMD hardware (a first for me).
3. **Early-Stop at 90%** 🚦: because I’m focusing on iterative improvement rather than blindly training for 200 epochs.

---

## Project Overview 📝

1. **Data**: [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html).  
   - 50k training images, 10k test images, 10 classes, each 32×32.
2. **Model**:  
   - Patch size = 4 (so each “patch” is 4×4),
   - Embedding dimension = 384,
   - 8 Transformer layers,
   - 6 attention heads.
3. **Training**:
   - Up to 50 epochs,
   - Batch size = 128,
   - My personal preference: Let’s see if we can get to ~85% accuracy and see how far the final model can push.

I love how flexible the ViT architecture is—it’s a perfect example of “attention-based” approaches crossing domains. 🧠💡

---

## About the AMD Instinct MI300X GPU 🔬

I’m running this on AMD Instinct MI300X, which is basically the HPC-oriented GPU from AMD. It's awesome for large-batch training because it has a ton of memory. Also:

- MI300X has advanced **ROC**m and BF16/FP16 training performance.
- I’m using PyTorch’s built-in support for ROCm.

*(If you’re on a different AMD GPU or HPC environment, you can adapt the code accordingly.)*

---

## Setup & Dependencies 🏗️

- **PyTorch** with ROCm 6.1 support (or whichever version you use).
- Basic Python libraries: `torchvision`, `tqdm`, `numpy`, `matplotlib`, etc.

Example installation snippet (though you may have your environment pre-configured):
```bash
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/rocm6.1
pip install tqdm matplotlib pandas
```

---

## Notebook Sections 📔

1. **Imports & Hyperparameters**  
   I declare all the key settings: patch size, embed dimension, batch size, epochs, etc.  
2. **Data Loaders**  
   - CIFAR-10 with RandAugment (2 ops, magnitude=9).  
   - My reason: RandAugment is quick to set up and can significantly boost accuracy on smaller datasets.
3. **Vision Transformer Implementation**  
   - `PatchEmbedding` for 32×32 → flatten → embed.  
   - Then standard MLP, multi-head attention blocks, etc.  
4. **Training Loop**  
   - Mixed Precision via `autocast(enabled=True)`.  
   - Early-stop if test accuracy ≥ 90%.  
5. **Inference Benchmark**  
   - I do a short warm-up, then measure how many images/sec the model can process at batch sizes 1,4,16,64,128.  
   - Because let’s face it, we all love seeing big throughput numbers!  
6. **Visualizations**  
   - I plot training curves in `training_metrics.png`.  
   - I also show random sample predictions in `sample_predictions.png`.

---

## Let’s Begin! 🎈

1. **Check GPU**: I like to do:
   ```python
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   print("[INFO] Device type:", device)
   if device.type == "cuda":
       print("[INFO] GPU name:", torch.cuda.get_device_name(0))
   ```
   Because I want to confirm I’m actually on the MI300X.

2. **Data**:  
   - `get_cifar10_loaders()`—it’s super simple: once you see “Files already downloaded,” you know you’re ready.

*(At this point, code in the following cells sets up transformations and data loaders.)*

---

## Training 🏋️‍♀️

- The code inside `train_one_epoch()` uses `autocast` with standard AMP.  
- I personally appreciate how PyTorch’s `GradScaler` helps avoid the usual half-precision pitfalls.

You’ll see a progress bar from **`tqdm`**:

```
Epoch 1/50: 100%|██████████| 391/391 [00:14<00:00, 27.42it/s, loss=1.9474, acc=26.60]
[Epoch 1/50] Train Acc=26.60% | Test Acc=39.68%
```

*(Yes, it’s only 26.60% after the first epoch, but that’s normal for a bigger architecture on CIFAR-10.)*

I log final training time around 13 minutes to get to ~84% best test accuracy.  I consider that quite efficient for a mild-scale model on a single GPU. 💪

---

## Inference & Scaling 🚀

- The code in `benchmark_inference()` runs batch sizes `[1,4,16,64,128]`.
- I warm up 10 iterations, then measure a certain number (20 or 50) of actual runs.
- Output example (which you’ll see in the notebook’s cell):
  ```
  [Inference] BS=1,  3.47 ms, 288.09 img/s
  [Inference] BS=4,  3.74 ms, 1068.95 img/s
  ...
  [Inference] BS=128, 6.25 ms, 20482.77 img/s
  ```

To me, these throughput numbers are quite satisfying. 📈💯

---

## My Personal Reflections 🤗

- I love how easy it is to integrate Transformers in PyTorch, especially for smaller images like 32×32.  
- Mixed precision definitely speeds up training on the MI300X while being stable.  
- Reaching 84% is quite decent for a moderate-sized ViT in ~13 minutes.  
- If I wanted to surpass 90%, I’d run more epochs or enlarge the network. But for now, I’m satisfied with these results as a baseline demonstration.

---

## Next Steps 🍀

1. **Longer Training**: Try 100+ epochs for potential 90%+ accuracy.  
2. **Bigger ViT**: Increase `EMBED_DIM` from 384 to 512 or 768.  
3. **More Data**: CIFAR-100 or a subset of ImageNet, for a real test.  
4. **Advanced Augment**: Could do Mixup or CutMix in synergy with RandAugment.

---

## Conclusion ✨

I hope walking through this notebook gave insight into:
- Setting up a Vision Transformer on AMD hardware,
- Achieving real-time speed improvements with AMP,
- And seeing a fun look at how Transformers handle small images.

**Thanks for checking out my personal notes!**  
Feel free to tweak the code or explore further. Let’s keep pushing those attention-based architectures to new frontiers!
```

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Device type:", device)
if device.type == "cuda":
    print("[INFO] GPU name:", torch.cuda.get_device_name(0))

[INFO] Device type: cuda
[INFO] GPU name: AMD Instinct MI300X


In [9]:
"""
MI300X CIFAR-10 Vision Transformer (FP16 via Standard AMP):
- 50 epochs (or early-stop at 90% accuracy)
- RandAugment for improved accuracy
- Mixed / half-precision using autocast(enabled=True)
- Prints GPU model name in console output
- Plots training metrics with "MI300X" explicitly mentioned
- Larger sample predictions figure so text isn't cut off
"""

import os
import time
import json
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import (
    Compose,
    ToTensor,
    Normalize,
    RandomCrop,
    RandomHorizontalFlip,
    RandAugment
)
from torch.cuda.amp import autocast, GradScaler
from datetime import datetime

matplotlib.use("Agg")  # For headless environments

# -------------------------------
# Hyperparameters & Config
# -------------------------------
BATCH_SIZE = 128
NUM_EPOCHS = 50
IMAGE_SIZE = 32
NUM_CLASSES = 10
PATCH_SIZE = 4
EMBED_DIM = 384
NUM_HEADS = 6
NUM_LAYERS = 8
MLP_RATIO = 4.0
DROPOUT = 0.1
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.03
SAVE_DIR = "results"

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Device Type: {device}")
if device.type == "cuda":
    # Print actual GPU name (ex: "AMD Instinct MI300X")
    real_gpu_name = torch.cuda.get_device_name(0)
    print(f"[INFO] GPU Model: {real_gpu_name}")

# ============================================
# 1) CIFAR-10 Dataloaders (RandAugment)
# ============================================
def get_cifar10_loaders(batch_size=128):
    transform_train = Compose([
        RandomCrop(32, padding=4),
        RandomHorizontalFlip(),
        RandAugment(num_ops=2, magnitude=9),  # for better accuracy
        ToTensor(),
        Normalize([0.4914,0.4822,0.4465],[0.2470,0.2435,0.2616])
    ])
    transform_test = Compose([
        ToTensor(),
        Normalize([0.4914,0.4822,0.4465],[0.2470,0.2435,0.2616])
    ])
    train_set = CIFAR10(root="data", train=True, download=True, transform=transform_train)
    test_set  = CIFAR10(root="data", train=False, download=True, transform=transform_test)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    return train_loader, test_loader

# ============================================
# 2) Vision Transformer Building Blocks
# ============================================
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=384):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.num_patches = (img_size // patch_size)**2

    def forward(self, x):
        x = self.conv(x)        # [B, embed_dim, H', W']
        x = x.flatten(2)        # [B, embed_dim, N]
        x = x.transpose(1, 2)   # [B, N, embed_dim]
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim  = embed_dim // num_heads

        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        self.attn_drop = nn.Dropout(dropout)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_drop = nn.Dropout(dropout)

    def forward(self, x):
        B, N, C = x.shape
        # [B, N, 3*embed_dim] -> [B, N, 3, heads, head_dim] -> rearrange
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # each: [B, heads, N, head_dim]

        attn_scores = (q @ k.transpose(-2, -1)) * (self.head_dim**-0.5)
        attn = attn_scores.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = (attn @ v).transpose(1, 2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop= nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn  = MultiHeadAttention(embed_dim, num_heads, dropout)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp   = MLP(embed_dim, int(embed_dim*mlp_ratio), embed_dim, dropout)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    def __init__(
        self,
        img_size=32,
        patch_size=4,
        in_channels=3,
        num_classes=10,
        embed_dim=384,
        depth=8,
        num_heads=6,
        mlp_ratio=4.0,
        dropout=0.1
    ):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_channels, embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches+1, embed_dim))
        self.pos_drop  = nn.Dropout(dropout)

        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

        self._init_weights()

    def _init_weights(self):
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        self.apply(self._init_weights_general)

    def _init_weights_general(self, module):
        if isinstance(module, nn.Linear):
            nn.init.trunc_normal_(module.weight, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.LayerNorm):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)       # => [B, N, E]
        cls = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls, x), dim=1)# => [B, N+1, E]

        x = x + self.pos_embed[:, :x.size(1), :]
        x = self.pos_drop(x)

        for blk in self.blocks:
            x = blk(x)

        x = self.norm(x)
        logits = self.head(x[:, 0])
        return logits

# ============================================
# 3) Train/Evaluate with Standard AMP
# ============================================
def train_one_epoch(model, loader, criterion, optimizer, device, scaler, epoch, total_epochs):
    model.train()
    correct, total, running_loss = 0, 0, 0.0
    start_time = time.time()

    pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{total_epochs}")
    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # Use standard AMP (no forced dtype)
        with autocast(enabled=True):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * targets.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

        acc = 100.0 * correct / total
        pbar.set_postfix({"loss": f"{running_loss/total:.4f}", "acc": f"{acc:.2f}"})

    epoch_time = time.time() - start_time
    return {
        "loss": running_loss/total,
        "accuracy": acc,
        "epoch_time": epoch_time,
        "images_per_sec": total/epoch_time
    }

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    correct, total, running_loss = 0, 0, 0.0

    for inputs, targets in tqdm(loader, desc="Evaluating", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)

        with autocast(enabled=True):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        running_loss += loss.item() * targets.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

    return {
        "loss": running_loss/total,
        "accuracy": 100.0 * correct/total
    }

def visualize_predictions(model, loader, device, num_images=5):
    """
    Plots a few random images from the test set with bigger figure.
    Moves the images down so the title isn't merged with them.
    """
    model.eval()
    data_iter = iter(loader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad(), autocast(enabled=True):
        outputs = model(images)
        _, preds = outputs.max(1)

    class_names = loader.dataset.classes

    fig, axes = plt.subplots(1, num_images, figsize=(5*num_images, 5))
    for i in range(num_images):
        ax = axes[i]
        idx = random.randint(0, len(images) - 1)
        img = images[idx].cpu().numpy().transpose(1,2,0)
        # Unnormalize
        mean = np.array([0.4914, 0.4822, 0.4465])
        std  = np.array([0.2470, 0.2435, 0.2616])
        img  = img*std + mean
        img  = np.clip(img, 0, 1)

        true_lbl = class_names[labels[idx].item()]
        pred_lbl = class_names[preds[idx].item()]
        ax.imshow(img)
        ax.set_title(f"True: {true_lbl}\nPred: {pred_lbl}", fontsize=11)
        ax.axis("off")
        ax.set_anchor("C")

    
    # Adjust the figure so there's enough space under the title
    plt.subplots_adjust(top=0.85)

    # Or you can keep tight_layout() then re-adjust
    # plt.tight_layout()
    # plt.subplots_adjust(top=0.80)

    out_png = os.path.join(SAVE_DIR, "sample_predictions.png")
    plt.savefig(out_png, dpi=200)
    plt.close()


@torch.no_grad()
def benchmark_inference(model, device, max_bs=128):
    """
    Benchmarks inference for [1,4,16,64,128], saves to 'inference_stats.json'.
    """
    model.eval()
    results = {}
    for bs in [1,4,16,64,128]:
        if bs > max_bs:
            break
        # We'll feed float32 inputs; autocast handles them
        dummy = torch.randn(bs, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)

        # Warmup
        for _ in range(10):
            with autocast(enabled=True):
                _ = model(dummy)
        torch.cuda.synchronize()

        # Benchmark
        start_t = time.time()
        iters = 50 if bs <= 16 else 20
        for _ in range(iters):
            with autocast(enabled=True):
                _ = model(dummy)
        torch.cuda.synchronize()
        dur = (time.time() - start_t)/iters

        results[str(bs)] = {
            "inference_time_ms": dur*1e3,
            "images_per_sec": bs/dur
        }
        print(f"[Inference] BS={bs}, {dur*1e3:.2f} ms, {bs/dur:.2f} img/s")

    out_path = os.path.join(SAVE_DIR,"inference_stats.json")
    with open(out_path, 'w') as f:
        json.dump(results, f, indent=2)
    print("[INFO] Inference stats saved:", out_path)

def plot_training_metrics(train_stats, test_stats):
    """
    Plots train/test loss & accuracy, with an MI300X mention in the title.
    """
    epochs = range(1, len(train_stats)+1)
    tr_loss= [s['loss'] for s in train_stats]
    tr_acc = [s['accuracy'] for s in train_stats]
    te_loss= [s['loss'] for s in test_stats]
    te_acc = [s['accuracy'] for s in test_stats]

    fig, axs = plt.subplots(1,2, figsize=(14,5))
    fig.suptitle("Training on AMD Instinct MI300X (50 Epochs)", fontsize=14)

    # Loss
    axs[0].plot(epochs, tr_loss, 'b-o', label='Train Loss')
    axs[0].plot(epochs, te_loss, 'r-s', label='Test Loss')
    axs[0].set_title("Loss")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].legend()
    axs[0].grid(True)

    # Accuracy
    axs[1].plot(epochs, tr_acc, 'b-o', label='Train Acc')
    axs[1].plot(epochs, te_acc, 'r-s', label='Test Acc')
    axs[1].set_title("Accuracy (%)")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout()
    out_fig = os.path.join(SAVE_DIR,"training_metrics.png")
    plt.savefig(out_fig, dpi=150)
    plt.close()

def main():
    print("[MAIN] Vision Transformer on CIFAR-10 (50 epochs, standard AMP)")
    train_loader, test_loader = get_cifar10_loaders(batch_size=BATCH_SIZE)

    model = VisionTransformer(
        img_size=IMAGE_SIZE,
        patch_size=PATCH_SIZE,
        in_channels=3,
        num_classes=NUM_CLASSES,
        embed_dim=EMBED_DIM,
        depth=NUM_LAYERS,
        num_heads=NUM_HEADS,
        mlp_ratio=MLP_RATIO,
        dropout=DROPOUT
    ).to(device)

    print(f"[MAIN] #parameters: {sum(p.numel() for p in model.parameters()):,}")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
    scaler    = GradScaler()

    train_stats= []
    test_stats = []
    best_acc   = 0.0

    start_all = time.time()
    for epoch in range(NUM_EPOCHS):
        # Train
        tr_stat = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler, epoch, NUM_EPOCHS)
        train_stats.append(tr_stat)

        # Evaluate
        ev_stat = evaluate(model, test_loader, criterion, device)
        test_stats.append(ev_stat)

        scheduler.step()

        print(f"[Epoch {epoch+1}/{NUM_EPOCHS}] Train Acc={tr_stat['accuracy']:.2f}% | "
              f"Test Acc={ev_stat['accuracy']:.2f}%")
        if ev_stat['accuracy']> best_acc:
            best_acc = ev_stat['accuracy']
        if ev_stat['accuracy']>= 90.0:
            print(f"[MAIN] Reached 90% test accuracy at epoch {epoch+1}, stopping early.")
            break

    total_time = time.time() - start_all
    print(f"[MAIN] Training took {total_time/60:.2f} minutes. Best test acc={best_acc:.2f}%")

    # Save model
    torch.save(model.state_dict(), "checkpoints/vit_final_mi300x.pth")
    print("[MAIN] Model checkpoint saved.")

    # Plot training
    plot_training_metrics(train_stats, test_stats)

    # Visualize sample predictions
    visualize_predictions(model, test_loader, device, num_images=5)

    # Benchmark inference
    benchmark_inference(model, device, max_bs=128)

    print("[MAIN] All tasks done. Check 'results/' for plots and 'checkpoints/' for model.")


if __name__ == "__main__":
    main()

[INFO] Device Type: cuda
[INFO] GPU Model: AMD Instinct MI300X
[MAIN] Vision Transformer on CIFAR-10 (50 epochs, standard AMP)
Files already downloaded and verified
Files already downloaded and verified
[MAIN] #parameters: 14,244,490


Epoch 1/50: 100%|██████████| 391/391 [00:13<00:00, 28.44it/s, loss=1.9477, acc=26.29]


[Epoch 1/50] Train Acc=26.29% | Test Acc=35.88%


Epoch 2/50: 100%|██████████| 391/391 [00:14<00:00, 27.83it/s, loss=1.6824, acc=37.80]


[Epoch 2/50] Train Acc=37.80% | Test Acc=42.75%


...

[Epoch 48/50] Train Acc=81.57% | Test Acc=83.04%


Epoch 49/50: 100%|██████████| 391/391 [00:14<00:00, 26.63it/s, loss=0.5154, acc=81.70]


[Epoch 49/50] Train Acc=81.70% | Test Acc=83.00%


Epoch 50/50: 100%|██████████| 391/391 [00:14<00:00, 27.05it/s, loss=0.5159, acc=81.65]


[Epoch 50/50] Train Acc=81.65% | Test Acc=82.96%
[MAIN] Training took 13.41 minutes. Best test acc=83.08%
[MAIN] Model checkpoint saved.
[Inference] BS=1, 3.47 ms, 288.10 img/s
[Inference] BS=4, 3.74 ms, 1068.96 img/s
[Inference] BS=16, 4.33 ms, 3697.96 img/s
[Inference] BS=64, 5.29 ms, 12087.71 img/s
[Inference] BS=128, 6.25 ms, 20482.77 img/s
[INFO] Inference stats saved: results/inference_stats.json
[MAIN] All tasks done. Check 'results/' for plots and 'checkpoints/' for model.


In [10]:
"""
MI300X Performance Analysis Script (No GPU Metrics)

- Loads performance_report.json (if present) for optional reference
- Loads inference_stats.json and creates a scaling efficiency plot
- Saves output plots to 'analysis/' directory

Usage:
  python analysis_mi300x.py
"""

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create an 'analysis' folder if it doesn't exist
os.makedirs("analysis", exist_ok=True)

def load_data():
    """
    Loads optional performance report from 'results/performance_report.json'
    and inference stats from 'results/inference_stats.json'.
    Returns a dict with keys:
      'report' (dict or None)
      'inference' (dict or None)
    """
    results = {}

    # 1) Attempt to load performance report
    report_path = 'results/performance_report.json'
    if os.path.isfile(report_path):
        try:
            with open(report_path, 'r') as f:
                results['report'] = json.load(f)
        except Exception as e:
            print(f"[Analysis] Could not load performance_report.json: {e}")
            results['report'] = None
    else:
        print("[Analysis] performance_report.json not found.")
        results['report'] = None

    # 2) Attempt to load inference stats
    inference_path = 'results/inference_stats.json'
    if os.path.isfile(inference_path):
        try:
            with open(inference_path, 'r') as f:
                results['inference'] = json.load(f)
        except Exception as e:
            print(f"[Analysis] Could not load inference_stats.json: {e}")
            results['inference'] = None
    else:
        print("[Analysis] inference_stats.json not found.")
        results['inference'] = None

    return results

def create_scaling_efficiency_plot(inference_data):
    """
    Creates a scaling efficiency plot based on 'inference_data',
    which should be a dict like:
      {
        "1": {"images_per_sec":..., "inference_time_ms":...},
        "4": {...},
        ...
      }
    Saves a 2-subplot figure to 'analysis/mi300x_throughput_scaling.png'.
    """
    if not inference_data:
        print("[Analysis] No inference data found; skipping scaling plot.")
        return

    # Convert to a DataFrame
    entries = []
    for bs_str, stats in inference_data.items():
        bs = int(bs_str)
        entries.append({
            'batch_size': bs,
            'throughput': stats['images_per_sec'],
            'latency_ms': stats['inference_time_ms']
        })
    inf_df = pd.DataFrame(entries).sort_values('batch_size')

    # If batch_size=1 is present, compute ideal throughput & scaling efficiency
    if (inf_df['batch_size'] == 1).any():
        base_throughput = inf_df.loc[inf_df['batch_size'] == 1, 'throughput'].values[0]
        inf_df['ideal_throughput'] = inf_df['batch_size'] * base_throughput
        inf_df['scaling_efficiency'] = (inf_df['throughput'] / inf_df['ideal_throughput']) * 100
    else:
        inf_df['ideal_throughput'] = None
        inf_df['scaling_efficiency'] = None
        print("[Analysis] No batch_size=1 in data -> skipping ideal scaling lines.")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle("MI300X Inference Scaling Efficiency", fontsize=14)

    # Left subplot: actual throughput vs. batch size
    ax1.plot(inf_df['batch_size'], inf_df['throughput'], 'bo-', label='Actual Throughput')
    if inf_df['ideal_throughput'].notna().any():
        ax1.plot(inf_df['batch_size'], inf_df['ideal_throughput'], 'r--', label='Ideal Linear')
    ax1.set_xscale('log', base=2)
    ax1.set_xlabel('Batch Size (log scale)')
    ax1.set_ylabel('Throughput (images/sec)')
    ax1.set_title('Throughput vs. Batch Size')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(inf_df['batch_size'])
    ax1.set_xticklabels([str(x) for x in inf_df['batch_size']])

    # Right subplot: scaling efficiency
    if inf_df['scaling_efficiency'].notna().any():
        ax2.plot(inf_df['batch_size'], inf_df['scaling_efficiency'], 'go-')
        ax2.set_xscale('log', base=2)
        ax2.set_xlabel('Batch Size (log scale)')
        ax2.set_ylabel('Scaling Efficiency (%)')
        ax2.set_title('Scaling Efficiency')
        ax2.grid(True, alpha=0.3)
        ax2.set_xticks(inf_df['batch_size'])
        ax2.set_xticklabels([str(x) for x in inf_df['batch_size']])
    else:
        ax2.text(0.5, 0.5, "No scaling efficiency (missing batch_size=1).",
                 ha='center', va='center', fontsize=12)
        ax2.set_title("Scaling Efficiency")

    plt.tight_layout()
    out_path = 'analysis/mi300x_throughput_scaling.png'
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"[Analysis] Saved inference scaling plot to {out_path}")

def main():
    """
    Main entry point for analysis. Loads the data, then plots the inference scaling.
    """
    data = load_data()

    # If you have a performance report, you could parse or print it here
    if data['report']:
        print("[Analysis] Found a performance report. You can parse or print it below:")
        # For example:
        # print(json.dumps(data['report'], indent=2))

    # Create scaling efficiency plot from inference stats
    create_scaling_efficiency_plot(data['inference'])

    print("[Analysis] Done. Check 'analysis/' folder for output plots.")

if __name__ == "__main__":
    main()


[Analysis] performance_report.json not found.
[Analysis] Saved inference scaling plot to analysis/mi300x_throughput_scaling.png
[Analysis] Done. Check 'analysis/' folder for output plots.
